# Algoritmos de búsqueda

Creado por:
* David Piñeiro López
* Gonzalo Molina Márquez
* Manuel Pasieka
* Óscar Piqueras Segura
* Samuel Eduardo Bermejo Bramley


El objetivo es desarrollar en lenguaje de programación Python una implementación del algoritmo para afianzar la mecánica de implementación de un primer algoritmo de búsqueda.

Deseamos resolver el problema: dada una configuración inicial del mundo de los bloques en la cual los cubos se encuentran como se indica en la figura, deseamos alcanzar una ordenación correcta de los mismos.

![figura1](./t6_act.jpg)


In [1]:
class State:
    ''' Esta clase tiene por objetivo el definir el estado
    Servirá para definir el estado inicial, el estado final,
    así como cada uno de los estados intermedios
    '''
    
    def __init__(self, stack, table, action):
        self.stack  = stack
        self.table  = table
        self.action = action
    
    def compare(self, state):
        ''' Esta función se encarga de comprobar si dos estados son iguales
        '''
        if (len(self.stack) != len(state.stack) | len(self.table) != len(state.table)):
            return False
        for i in range(len(state.stack)):
            if self.stack[i] != state.stack[i]:
                return False
        return len(set(self.table) & set(state.table)) == len(state.table)
    
    def heuristic(self, state):
        ''' Calcula la distancia al estado final
        '''
        state_len = len(state.stack)
        self_len  = len(self.stack)
        for i in range(self_len):
            if self.stack[i] != state.stack[i]:
                return (state_len - i) + (self_len - i)
        return state_len - self_len
    
    def pop(self):
        ''' Saca del stack, es decir, pone encima de la mesa,
        la última pieza, la situada en la cima de la columna
        '''
        piece = self.stack.pop()
        self.table.append(piece)
        return piece
        
    def push(self, piece):
        ''' Pone una pieza de la mesa en el stack (columna de
        piezas)
        '''
        self.table.remove(piece)
        self.stack.append(piece)
        return
    
    def copy(self, action):
        ''' Realiza una copia del estado dada una acción
        '''
        state = State(
            self.stack.copy(),
            self.table.copy(),
            action)
        return state
        
    def expand(self):
        ''' 
        '''
        expand_list = []
        for piece in self.table:
            state = self.copy("push {0} in stack".format(piece))
            state.push(piece)
            expand_list.append(state)

        if (len(self.stack) != 0):
            state = self.copy(None)      
            state.action = "pop {0} from stack".format(state.pop())
            expand_list.append(state)
        
        return expand_list
    
    def print_state(self):
        ''' Saca por pantalla los valores del estado
        '''
        print('Acción {}: Stack {}: Table {}'.format(self.action, self.stack, self.table))

In [2]:
class Node:
    '''Esta clase tiene por objetivo el definir el estado
    Servirá para definir el estado inicial, el estado final,
    así como cada uno de los estados intermedios
    '''
    
    def __init__(self, state, father_node, cost = 0, heuristic = 9999999):
        self.state       = state 
        self.father_node = father_node
        self.cost        = cost
        self.heuristic   = heuristic
        self.f           = cost + heuristic
        
    def printNode(self):
        self.state.print_state()
        
    def print_family(self):
        '''Esta función saca por pantalla la rama del arbol
        perteneciente al Nodo
        '''
        l = []
        n = self
        while n != None:
            l.append(n)
            n = n.father_node
        l.reverse()
        for n in l:
            n.state.print_state()

    def isBucle(self, state):
        ''' Comprueba si el estado pasado por argumento ya ha
        sido usado con anterioridad en la rama actual.
        '''
        n = self
        while n != None:
            if n.state.compare(state):
                return True
            n = n.father_node
        return False

In [5]:
class general_search:
    
    def __init__(self, state, goal_state):
        self.state      = state
        self.goal_state = goal_state
        assert len(set(state.stack + state.table)) == len(set(goal_state.stack + goal_state.table)), 'El estado inicial y el estado final son de distinto tamaño'
        
        
    def print_performance(self, visited_nodes, nodes_open):
        ''' Esta función saca por pantalla información relevante
        al algoritmo de búsqueda empleado
        '''
        print('Nodos en abierta = {}'.format(nodes_open))            
        print('Nodos visitados  = {}\n'.format(visited_nodes))
        return
        
    def search_breadth_first(self):
        print('Búsqueda en Amplitud')
        init_time = time()
        node      = Node(self.state, father_node=None)
        open_list = [node]
        visited_nodes = 0

        while True:
            assert len(open_list) != 0, 'Se han explorado todas las opciones sin exito'
            node = open_list.pop(0)

            if node.state.compare(self.goal_state):
                # Hemos alcanzado el estado final
                final_time = time()
                self.print_performance(visited_nodes,len(open_list))
                node.print_family()
                print('Tiempo de ejecución: {} s\n'.format(final_time - init_time))
                break
            
            else:
                # Añadimos todos los posibles hijos a la lista a explorar
                visited_nodes += 1
                succesors = node.state.expand()
                for state in succesors:
                    if not node.isBucle(state):
                        n = Node(state=state, father_node=node)
                        open_list.append(n)
        return node

    def search_depth_first(self):
        print('Búsqueda en Profundidad')
        init_time = time()
        node      = Node(self.state, father_node=None)
        open_list = [node]
        visited_nodes = 0

        while True:
            assert len(open_list) != 0, 'Se han explorado todas las opciones sin exito'
            node = open_list.pop()

            if node.state.compare(self.goal_state):
                # Hemos alcanzado el estado final
                final_time = time()
                self.print_performance(visited_nodes,len(open_list))
                node.print_family()
                print('Tiempo de ejecución: {} s\n'.format(final_time - init_time))
                break
            
            else:
                # Añadimos todos los posibles hijos a la lista a explorar
                visited_nodes += 1
                succesors = node.state.expand()
                for state in succesors:
                    if not node.isBucle(state):
                        n = Node(state=state, father_node=node)
                        open_list.append(n)
        return node
    
    def search_A_star(self):
        print('Búsqueda A*')
        init_time = time()
        node      = Node(self.state, father_node=None)
        open_list = [node]
        visited_nodes = 0

        while True:            
            assert len(open_list) != 0, 'Se han explorado todas las opciones sin exito'
            node = open_list.pop(0)

            if node.state.compare(self.goal_state):
                # Hemos alcanzado el estado final
                final_time = time()
                self.print_performance(visited_nodes,len(open_list))
                node.print_family()
                print('Tiempo de ejecución: {} s\n'.format(final_time - init_time))
                break
            
            else:
                # Añadimos todos los posibles hijos a la lista a explorar
                # Es necesario calcular el coste y el heuristic para los nodos hijos
                # Ordenamos la lista en función de los valores de heuristic
                visited_nodes += 1
                succesors = node.state.expand()
                for state in succesors:
                    if not node.isBucle(state):
                        h = state.heuristic(self.goal_state)
                        g = node.cost + 1
                        n = Node(
                            state       = state,
                            father_node = node,
                            cost        = g,
                            heuristic   = h)
                        open_list.append(n)
                open_list = sorted(open_list, key = lambda x: x.f)

        return node         

    def search_hill_climbing(self):
        print('Búsqueda Hill Climbing')
        init_time = time()
        node      = Node(self.state, father_node=None)
        open_list = [node]
        visited_nodes = 0

        while True:
            assert len(open_list) != 0, 'Se han explorado todas las opciones sin exito'
            node = open_list.pop(0)

            if node.state.compare(self.goal_state):
                # Hemos alcanzado el estado final
                final_time = time()
                self.print_performance(visited_nodes,len(open_list))
                node.print_family()
                print('Tiempo de ejecución: {} s\n'.format(final_time - init_time))
                break
            
            else:
                visited_nodes += 1
                succesors = node.state.expand()
                succ = []
                for state in succesors:
                    h = state.heuristic(self.goal_state)
                    g = node.cost + 1
                    n = Node(
                        state       = state,
                        father_node = node,
                        cost        = g,
                        heuristic   = h)
                    succ.append(n)
                succ = sorted(succ,key= lambda x: x.f)
                open_list.append(succ.pop(0))
        
        return node

In [7]:
from time import time

initial_state = State(['E','D','A'],['C','B'], 'Inicio')
goal_state    = State(['E','D','C','B','A'],[],'Estado Final')

s = general_search(initial_state,goal_state)

result = s.search_breadth_first()
result = s.search_depth_first()
result = s.search_A_star()
result = s.search_hill_climbing()

Búsqueda en Amplitud
Nodos en abierta = 20
Nodos visitados  = 19

Acción Inicio: Stack ['E', 'D', 'A']: Table ['C', 'B']
Acción pop A from stack: Stack ['E', 'D']: Table ['C', 'B', 'A']
Acción push C in stack: Stack ['E', 'D', 'C']: Table ['B', 'A']
Acción push B in stack: Stack ['E', 'D', 'C', 'B']: Table ['A']
Acción push A in stack: Stack ['E', 'D', 'C', 'B', 'A']: Table []
Tiempo de ejecución: 0.0007596015930175781 s

Búsqueda en Profundidad
Nodos en abierta = 2
Nodos visitados  = 95423

Acción Inicio: Stack ['E', 'D', 'A']: Table ['C', 'B']
Acción pop A from stack: Stack ['E', 'D']: Table ['C', 'B', 'A']
Acción push C in stack: Stack ['E', 'D', 'C']: Table ['B', 'A']
Acción push B in stack: Stack ['E', 'D', 'C', 'B']: Table ['A']
Acción push A in stack: Stack ['E', 'D', 'C', 'B', 'A']: Table []
Tiempo de ejecución: 3.028024673461914 s

Búsqueda A*
Nodos en abierta = 5
Nodos visitados  = 4

Acción Inicio: Stack ['E', 'D', 'A']: Table ['C', 'B']
Acción pop A from stack: Stack ['E', 

# Conclusiones

Se han implementado cuatro algoritmos de búsqueda entre los que podemos encontrar la búqueda por amplitud, búsqueda en profundidad, A star y Hill Climbing, siendo este último el algoritmo que encuentra la solución en el menor tiempo.
El algoritmo A star también ha demostrado tener muy buenos resultados. No obstante, la busqueda en profundidad ha sido la menos satisfactoria